# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [1]:
import requests
import pandas as pd
import os 

In [2]:
bikes_data = pd.read_csv('citybikes_PL.csv', index_col=0)
bikes_data

url = "https://api.foursquare.com/v3/places/search"
api_key = os.environ["APIfoursquare"]
headers = {"Accept": "application/json","Authorization": api_key}
response_dict = {}
for i in bikes_data.index:
    row = bikes_data.loc[i]
    lat_long = str(row['latitude']) + ',' + str(row['longitude'])
    params = {'radius':1000,'ll':lat_long}
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    response_dict[lat_long] = data

In [ ]:
response_dict

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
name_list = []
main_category_list = []
location_list = []
distance_list = []
lat_long_list = []
error_list_indexes = []
index_count = 0
for i in response_dict:
    for x in response_dict[i]['results']:
        lat_long_list.append(i)
        name_list.append(x['name'])
        distance_list.append(x['distance'])
        #there are 5 businesses with no category, so when that is the case it is replaced with 'n/a'
        try:
            main_category_list.append(x['categories'][0]['name'])
        except:
            main_category_list.append('n/a')
            error_list_indexes.append(index_count)
        index_count += 1
        #there are 31 businesses with no pirmary address, so when that is the case it is replaced with the formatted_address
        #since that will still give some information on the location even without the primary address
        try:
            location_list.append(x['location']['address'])
        except:
            location_list.append(x['location']['formatted_address'])
            error_list_indexes.append(index_count)
        index_count += 1

df_dict_fs = {'bike_station_location':lat_long_list, 'name':name_list, 'main_category':main_category_list, 'address':location_list, 'distance_away':distance_list}

#print(len(location_list), len(name_list), len(lat_long_list), len(main_category_list), len(distance_list))

Put your parsed results into a DataFrame

In [ ]:
df_fs = pd.DataFrame(df_dict_fs)
df_fs
#exporting to csv file for future use
#df_fs.to_csv('foursquare.csv', index=False)


bike_station_location	name	main_category	address	distance_away
0	45.51195046,-122.6141596	Por Que No	American Restaurant	4635 SE Hawthorne Blvd	21
1	45.51195046,-122.6141596	East Side Delicatessen	Deli	4626 SE Hawthorne Blvd	28
2	45.51195046,-122.6141596	Zach's Shack	Hot Dog Joint	4611 SE Hawthorne Blvd	55
3	45.51195046,-122.6141596	Jaciva's	Bakery	4733 SE Hawthorne Blvd	60
4	45.51195046,-122.6141596	Apizza Scholls	Pizzeria	4741 SE Hawthorne Blvd	82
...	...	...	...	...	...
2348	45.5002094,-122.670727	Starbucks	Coffee Shop	2730 S Moody Ave	351
2349	45.5002094,-122.670727	Frank Wine and Flower	Wine Bar	3712 S Bond Ave	445
2350	45.5002094,-122.670727	Flying Elephants	American Restaurant	2730 S Moody Ave	305
2351	45.5002094,-122.670727	South Waterfront Greenway	Landmarks and Outdoors	3715 S River Pkwy	422
2352	45.5002094,-122.670727	South Waterfront Farmers Market	Farmers Market	3508 S Moody Ave	344
2353 rows × 5 columns

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [6]:
import requests
import pandas as pd
import os

In [25]:
bikes_data = pd.read_csv('citybikes_PL.csv', index_col=0)
bikes_data

url = "https://api.yelp.com/v3/businesses/search"
api_key = os.environ["YELP_API"]
headers = {"accept": "application/json",'Authorization': 'Bearer %s' % api_key}
response_dict_yelp = {}
for i in bikes_data.index:
    row = bikes_data.loc[i]
    lat = row['latitude']
    long = row['longitude']
    lat_long = str(row['latitude']) + ',' + str(row['longitude'])
    params = {'radius':1000,'latitude':lat,'longitude':long }
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    response_dict_yelp[lat_long] = data

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
name_list = []
main_category_list = []
location_list = []
distance_list = []
lat_long_list = []
rating_list = []
review_count_list = []
status_list = []
price_list = []
error_list_indexes = []
index_count = 0
for i in response_dict_yelp:
    for x in response_dict_yelp[i]['businesses']:
        lat_long_list.append(i)
        name_list.append(x['name'])
        main_category_list.append(x['categories'][0]['title'])
        location_list.append(x['location']['display_address'][0])
        distance_list.append(x['distance'])
        rating_list.append(x['rating'])
        review_count_list.append(x['review_count'])
        if x['is_closed'] == False:
            status_list.append('Open')
        else:
            status_list.append('Closed')
        #there are 530 businesses without a price description, so those were replaced with 'n/a'
        try:
            price_list.append(x['price'])
        except:
            price_list.append('n/a')
            error_list_indexes.append(index_count)
        index_count += 1

df_dict_yelp = {'bike_station_location':lat_long_list, 'name':name_list, 'main_category':main_category_list, 'address':location_list, 
                'price':price_list, 'rating':rating_list, 'review_count':review_count_list, 'status':status_list, 'distance_away':distance_list}
#len(error_list_indexes)

Put your parsed results into a DataFrame

In [ ]:
df_yelp = pd.DataFrame(df_dict_yelp)
df_yelp
#exporting to csv file for future use
#df_yelp.to_csv('yelp.csv', index=False)


bike_station_location	name	main_category	address	price	rating	review_count	status	distance_away
0	45.51195046,-122.6141596	Baka Umai	Noodles	4703 SE Hawthorne Blvd	n/a	4.5	164	Open	31.189958
1	45.51195046,-122.6141596	Por Qué No? Taqueria	Mexican	4635 SE Hawthorne Blvd	$$	4.0	1357	Open	21.912464
2	45.51195046,-122.6141596	Apizza Scholls	Pizza	4741 SE Hawthorne Blvd	$$	4.0	1735	Open	82.518277
3	45.51195046,-122.6141596	Sapphire Hotel	Lounges	5008 SE Hawthorne Blvd	$$	4.5	487	Open	225.793673
4	45.51195046,-122.6141596	Khao Thai	Thai	4604 SE Hawthorne Blvd	n/a	4.5	72	Open	65.891309
...	...	...	...	...	...	...	...	...	...
4710	45.5002094,-122.670727	Ross Island Grocery & Cafe	Cafes	3502 SW Corbett Ave	$	4.0	121	Open	471.402251
4711	45.5002094,-122.670727	Olia Poke & Tea	Poke	2720 S Moody Ave	$$	4.5	101	Open	427.257395
4712	45.5002094,-122.670727	Frank Wine Bar	Wine Bars	3712 S Bond Ave	$$	4.5	71	Open	441.418806
4713	45.5002094,-122.670727	Oracle Coffee Company	Coffee & Tea	3875 Sw Bond Ave	$	4.5	149	Open	560.689569
4714	45.5002094,-122.670727	The Old Spaghetti Factory	Italian	715 S Bancroft St	$$	3.0	469	Open	771.190320
4715 rows × 9 columns

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

The data from the Yelp API was notably more comprehensive, providing an extensive business details—particularly pertinent ones such as ratings, review counts, and pricing. Moreover, the Yelp API offered a substantially larger number of businesses in comparison to the FourSquare API. This disparity might stem from the Yelp API's capacity to dynamically adjust the search radius based on the area's density, or possibly from Yelp hosting a broader array of business data. Furthermore, the Yelp API provided precise distance measurements, while the FourSquare API presented distances rounded to the nearest value.

Get the top 10 restaurants according to their rating

In [ ]:
df_yelp.sort_values('rating', ascending=False).head(10)


bike_station_location	name	main_category	address	price	rating	review_count	status	distance_away
3541	45.5590435,-122.6340066	Thom Portland	Vietnamese	[3039 NE Alberta St, Portland, OR 97211]	$$	5.0	46	Open	18.643061
3230	45.5554316,-122.5890544	Fresh Love	Juice Bars & Smoothies	[7434 NE Fremont St, Portland, OR 97213]	n/a	5.0	26	Open	855.834080
3343	45.4994679,-122.6854443	Thai Yummy	Street Vendors	[1010 SW Gibbs, Portland, OR 97239]	$	5.0	152	Open	405.643942
4112	45.5337829,-122.5788583	DZO Bar & Grill	Bars	[7907 NE Glisan St, Portland, OR 97213]	n/a	5.0	9	Open	857.210069
4516	45.4941194,-122.5786536	Taste of Afghan	Afghan	[3905 SE 82nd Ave, Eastport Food Center, Portl...	n/a	5.0	36	Open	67.196800
3737	45.55862047843104,-122.5638759	Dannon YoCream International	Desserts	[5858 NE 87th Ave, Portland, OR 97220]	n/a	5.0	1	Open	1012.829897
1112	45.49429475,-122.6719048	Su Thai Kitchen	Food Trucks	[3714 sw Macadam Ave, Portland, OR 97239]	n/a	5.0	21	Open	218.927617
2542	45.54179953,-122.6614845	Cliff’s PDX	Dive Bars	[128 NE Russell St, Portland, OR 97212]	$$	5.0	39	Open	192.098007
1115	45.49429475,-122.6719048	Delish Kitchen 108	Bubble Tea	[664 S Gaines St, Portland, OR 97239]	n/a	5.0	5	Open	225.456026
1116	45.49429475,-122.6719048	Mama's Shawarma	Mediterranean	[S Moody And Gaines, Portland, OR 97239]	n/a	5.0	8	Open	207.678